In [1]:
%autosave 0
# %matplotlib inline
%matplotlib notebook

from __future__ import division, print_function
import numpy as np
from tensorflow.python.client import device_lib
from keras import backend as K
import tensorflow as tf
import keras
from keras.datasets import cifar10


print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Autosave disabled
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11758168762817767733
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 175767552
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 1152167410082497691
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:06:00.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 172490752
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 4511128467210666188
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:07:00.0, compute capability: 3.7"
]


Using TensorFlow backend.


In [7]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten
from keras.models import Model
from keras import optimizers
# from keras.utils import multi_gpu_model

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
input_train_shape = x_train.shape[1:]

n_out = 10
n_fc1 = 256

input_layer = Input(shape=(input_train_shape), name='input')
conv1 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_1')(input_layer)
bn1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_1')(conv1)
pool1 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_1')(bn1)
conv2 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_2')(pool1)
bn2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_2')(conv2)
pool2 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_2')(bn2)
conv3 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_3')(pool2)
bn3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_3')(conv3)
pool3 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool3')(bn3)
conv4 = Conv2D(1024, (4,4), strides=(1,1), padding='valid', activation='relu', name='conv_4')(pool3)
bn4 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_4')(conv4)
flat1 = Flatten(name='flat_1')(bn4)
fc5 = Dense(1024, activation='relu', name='fc_5')(flat1)
bn5 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_5')(fc5)
fc6 = Dense(256, activation='relu', name='fc_6')(bn5)
bn6 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_6')(fc6)
output_layer = Dense(10, activation='softmax', name='fc_7')(bn6)

sgd = optimizers.SGD(lr=2, decay=1e-6, momentum=0.9)
model = Model(input_layer, output_layer)
# pmodel = multi_gpu_model(model)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 32, 32, 64)        256       
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 16, 16, 64)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 16, 16, 128)       73856     
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 16, 16, 128)       512       
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 8, 8, 128)         0         
__________

In [3]:
print (x_train.shape)
print (y_train.shape)

# scale intensities between 0 and 1
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('min:', x_train[0].min())
print('max:', x_train[0].max())


(50000, 32, 32, 3)
(50000, 1)
min: 0.0
max: 1.0


In [8]:
n_epochs = 5
n_batch_size = 4
labels_train = keras.utils.to_categorical(y_train, n_out)
labels_test = keras.utils.to_categorical(y_test, n_out)

Cb = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0)]


history = model.fit(x_train, labels_train,
                    epochs=n_epochs,
                    batch_size=n_batch_size,
                    callbacks=Cb)

evaluation = model.evaluate(x_test, labels_test, verbose=0)

print('\n[Train parameters]')
for item in history.params:
    print(item+':', history.params[item])
    
print('\n[Model evaluation]')
print('test', history.params['metrics'][0], format(np.mean(evaluation[0]), '.4f'))
print('test', history.params['metrics'][1], format(np.mean(evaluation[1]), '.4f'))

Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[4,4,256,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_1/SGD/mul_24 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](SGD_3/momentum/read, training_1/SGD/Variable_12/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op u'training_1/SGD/mul_24', defined at:
  File "/home/python/2.7/envs/tf1.8/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/python/2.7/envs/tf1.8/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/python/2.7/envs/tf1.8/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/python/2.7/envs/tf1.8/lib/python2.7/site-packages/traitlets/config/appl

In [12]:
model.save('cifar_feedforward_180711.h5')